# Reinforcement learning for Taxis

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [1]:
#This code is importing several libraries: gym, IPython.display, time, numpy, and random. 
#The gym library is commonly used for reinforcement learning tasks, providing a collection of environments to test and train agents.
#The IPython.display library is used for displaying output in the IPython environment, which allows for better visualization of results.
#The time library is used for time-related operations, such as pausing the program for a certain duration.
#The numpy library is used for numerical computations and array manipulation.
#The random library is used for generating random numbers or making random choices.
import gym
from IPython.display import clear_output
from time import sleep
import numpy as np
import random

In [28]:
#The code creates an instance of the "Taxi-v3" environment from the OpenAI Gym library using the gym.make() function. 
#The env variable represents this environment.
env = gym.make("Taxi-v3").env
env.render()

# reset environment to a new, random state
env.reset() 
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [29]:
# Environment Parameters
actions = {0: 'South', 1: 'North', 2: 'East', 3: 'West', 4: 'Pickup', 5: 'Dropoff'}
passenger = {'R': 0, 'G': 1, 'Y': 2, 'B': 3, 'Car': 4}
dest = {'R': 0, 'G': 1, 'Y': 2, 'B': 3}
# Reward: each_step=-1, incorrect_pickup_dropoff=-10, correct_dropoff=20
# Penalty: incorrect_pickup_dropoff=1

### Functions

In [30]:
# Brute Force Approach
def brute_force(state):
    env.s = state
    epochs = 0
    penalties, reward = 0, 0
    frames = [] # for animation
    # initial state
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': '--',
        'reward': '--'
        }
    )
    done = False
    
    while not done:
        # Next action is choosen randomly
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1
    return frames, epochs, penalties

In [31]:
# Visualise frames
def print_frames(frames, time_btw_frames=0.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(time_btw_frames)

In [32]:
# Train agent to find path using Reinforcement Learning
def train_agent():
    q_table = np.zeros([env.observation_space.n, env.action_space.n])
    # Hyperparameters
    # Learning rate
    alpha = 0.1
    # Discount factor: importance to future rewards
    gamma = 0.6
    # Exploration rate
    epsilon = 0.1

    # For plotting metrics
    all_epochs = []
    all_penalties = []

    for i in range(1, 100001):
        state = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info = env.step(action) 

#           Q-Learning
            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n")
    return q_table

In [33]:
# Run agent trained using Reinforcemnt learning
def run_agent(state, q_table):
    epochs = 0
    penalties, reward = 0, 0
    env.s = state
    
    frames = [] # for animation
    # initial state
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': '--',
        'reward': '--'
        }
    )
    done = False

    while not done:
        # Next action is choosen from q_table
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1
    return frames, epochs, penalties

### Initial State

In [34]:
# Loading state
# (taxi row, taxi column, passenger index, destination index)
# Blue letter indicates pickup location
# Pink Letter indicates dropoff location
state = env.encode(4, 4, passenger['Y'], dest['R']) 
env.s = state

# Uncomment next line to load random state
# env.reset()
state = env.s
print("State:", state)
env.render()

State: 488
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [35]:
# [Probability, Next State, Reward, Done] for the initial state
env.P[state]

{0: [(1.0, 488, -1, False)],
 1: [(1.0, 388, -1, False)],
 2: [(1.0, 488, -1, False)],
 3: [(1.0, 468, -1, False)],
 4: [(1.0, 488, -10, False)],
 5: [(1.0, 488, -10, False)]}

### Brute Force Approach

In [36]:
frames, epochs, penalties = brute_force(state)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 351
Penalties incurred: 121


In [37]:
# Visualise agent
print_frames(frames, time_btw_frames=0.03)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 351
State: 0
Action: 5
Reward: 20


## Reinforcement Learning

In [38]:
%%time
# Train Agent on 100000 episodes
q_table = train_agent()

Episode: 100000
Training finished.

CPU times: total: 43.5 s
Wall time: 1min 12s


In [39]:
# Printing the best action for our initial state
env.s = state
env.render()
print(q_table[state])
print("Best action: {}".format(actions[np.argmax(q_table[state])]))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
[-2.47607781 -2.47061344 -2.47604657 -2.47061344 -8.76739532 -9.26424737]
Best action: North


In [40]:
# Running agent trained using Reinforcement Learning
frames, epochs, penalties = run_agent(state, q_table)
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 14
Penalties incurred: 0


In [41]:
print_frames(frames,time_btw_frames=0.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 14
State: 0
Action: 5
Reward: 20


## Comparing Brute Force and Reinforement Learning approach

In [42]:
%%time
episodes = 100
brute_steps = 0
brute_penalties = 0
reinf_steps = 0
reinf_penalties = 0

for _ in range(episodes):
    env.reset()
    state = env.s
    
    frames, steps, penalties = brute_force(state)
    brute_steps += steps
    brute_penalties += penalties
    
    frames, steps, penalties = run_agent(state, q_table)
    reinf_steps += steps
    reinf_penalties += penalties
    
print('Result after {} episodes'.format(episodes))
print('Average Brute-Force timestamps per episode           : {}'.format(brute_steps/episodes))
print('Average Brute-Force penalties per episode            : {}'.format(brute_penalties/episodes))
print('Average Reinforcement Learning timestamps per episode: {}'.format(reinf_steps/episodes))
print('Average Reinforcement Learning penalties per episode : {}'.format(reinf_penalties/episodes))

Result after 100 episodes
Average Brute-Force timestamps per episode           : 2129.73
Average Brute-Force penalties per episode            : 690.29
Average Reinforcement Learning timestamps per episode: 13.35
Average Reinforcement Learning penalties per episode : 0.0
CPU times: total: 11.8 s
Wall time: 13.7 s


<p><b>This clearly indicates that Reinforcement Learning is much better than Brute Force, 
    because in the latter approach the agent is not learning from past experiences.</b></p>